In [1]:
from os.path import isfile, join
from os import listdir
import tempfile
from pdf2image import convert_from_path, convert_from_bytes
import time
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

## Convert pdf files to images


# Prepare Data For CNN

In [2]:
badImages = ["badImages/" + f for f in listdir('badImages')] #array with names of a bad images
goodImages = ["goodImages/" + f for f in listdir('goodImages')] #array with names of a good images

In [3]:
badImagesDataset = pd.DataFrame({
    'Name of a file': badImages ,
    'GoodOrBad': [0] * len(badImages)
})

In [4]:
goodImagesDataset = pd.DataFrame({
    'Name of a file': goodImages ,
    'GoodOrBad': [1] * len(goodImages),
})

In [5]:
dataset = badImagesDataset.append(goodImagesDataset) #Creating final dataset with all images

In [6]:
dataset.sample(frac = 1).reset_index(drop = True) #Shuffle dataset 

,Name of a file,GoodOrBad
0,goodImages/Estera_Katy Chow_Cosec.jpg,1
1,badImages/Man Loong_Brian Wong_Acc Asst.jpg,0
2,badImages/Rodyk_Nelson Lee_Sr Cosec.jpg,0
3,badImages/Zhenghe Capital_Rainbow Chen_Office ...,0
4,badImages/Tpaay_Farrahdiana_Admin.jpg,0
...,...,...
95,goodImages/UBS_Shawn Koh_CDD Assurance.jpg,1
96,goodImages/Cheng_Cheng_Lee Cho Kwan_Sr Cosec.jpg,1
97,goodImages/Vincent Chueng_Winnie Wong_Cosec As...,1
98,goodImages/Prophet_Maggie Chuang_Exec Sec.jpg,1


In [ ]:
X = dataset["Name of a file"]
y = dataset["GoodOrBad"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

In [ ]:
X_train[0]

In [ ]:
from shutil import copyfile
import glob
import os
from tqdm import tqdm

files = glob.glob('train/*')
for f in files:
    os.remove(f)
    
files = glob.glob('test/*')
for f in files:
    os.remove(f)

for source in X_train:
    copyfile(source, "train/" + source.split('/')[1])
    
for source in X_test:
    copyfile(source, "test/" + source.split('/')[1])

In [ ]:
#import cv2 

TRAIN_DIR = 'train'
TEST_DIR = 'test'
IMG_SIZE = 100

LR = 1e-5

MODEL_NAME = 'dogsvscats-{}-{}.model'.format(LR, '6conv-basic') 

In [ ]:
def label_img(img):
    if img in [f.split('/')[1] for f in goodImages]:
        return [1, 0]
    elif img in [f.split('/')[1] for f in badImages]:
        return [0, 1]

In [ ]:
import tqdm as tqdm
import os
import cv2

training_data = []

for img in os.listdir(TRAIN_DIR):
    label = label_img(img)
    path = os.path.join(TRAIN_DIR, img)
    img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE))
    training_data.append([np.array(img), np.array(label)])

np.save('train_data.npy', training_data)


In [ ]:
import tqdm as tqdm
import os
import cv2

testing_data = []

for img in os.listdir(TEST_DIR):
    label = label_img(img)
    path = os.path.join(TEST_DIR, img)
    img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE))
    testing_data.append([np.array(img), np.array(label)])

np.save('test_data.npy', testing_data)

In [ ]:
import tensorflow as tf
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

tf.reset_default_graph()

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir = 'log')


In [ ]:
train_data = np.load('train_data.npy', allow_pickle=True)
test_data = np.load('test_data.npy', allow_pickle=True)

In [ ]:
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

In [ ]:
X = np.array([i[0] for i in train_data]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train_data]

test_x = np.array([i[0] for i in test_data]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test_data]

In [ ]:
model.fit({'input': X}, {'targets': Y}, n_epoch=5, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

In [ ]:
model.save(MODEL_NAME)

In [ ]:
import matplotlib.pyplot as plt

# if you need to create the data:
#test_data = process_test_data()
# if you already have some saved:
test_data = np.load('test_data.npy', allow_pickle = True)

fig=plt.figure()

for num,data in enumerate(test_data[:2]):
    # cat: [1,0]
    # dog: [0,1]
    
    img_num = data[1]
    img_data = data[0]
    
    y = fig.add_subplot(1,2,num+1)
    orig = img_data
    data = img_data.reshape(IMG_SIZE,IMG_SIZE,1)
    #model_out = model.predict([data])[0]
    model_out = model.predict([data])[0]
    
    if np.argmax(model_out) == 1: str_label='Good'
    else: str_label='Bad'
        
    y.imshow(orig,cmap='gray')
    plt.title(str_label)
    y.axes.get_xaxis().set_visible(False)
    y.axes.get_yaxis().set_visible(False)
plt.show()